In [9]:
import shap

In [8]:
# Setup and installations - Enhanced
!pip install -q lightgbm==4.3.0 xgboost optuna==3.6.0 cmake
!apt-get -qq update && apt-get -qq install -y plink

import pandas as pd
import numpy as np
import os
import urllib.request
import sys
import warnings
from pathlib import Path
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (classification_report, roc_auc_score, roc_curve,
                           precision_recall_curve, average_precision_score, brier_score_loss, f1_score)
from sklearn.decomposition import PCA
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.inspection import partial_dependence, PartialDependenceDisplay

# Enhanced imports
import lightgbm as lgb
import xgboost as xgb
import shap
import matplotlib.pyplot as plt
import seaborn as sns

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [11]:
warnings.filterwarnings('ignore')
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [ ]:


# Data download configuration (unchanged)
BASE = "https://biobank.ndph.ox.ac.uk/synthetic_dataset/tabular/"
files_to_fetch = {
    "integer_no_arrays.tsv": BASE + "integer_no_arrays.tsv",
    "integer_diet_quest_fields.tsv": BASE + "integer_diet_quest_fields.tsv",
    "integer_other_quest_fields.tsv": BASE + "integer_other_quest_fields.tsv",
    "real_fields1.tsv": BASE + "real_fields1.tsv",
    "real_fields2.tsv": BASE + "real_fields2.tsv",
    "string_fields1.tsv": BASE + "string_fields1.tsv",
}

DATA_DIR = Path("/content/ukb_syn")
DATA_DIR.mkdir(exist_ok=True)

def fetch_data(url, out_path):
    """Download data files if they don't exist"""
    if out_path.exists():
        return
    print(f"⬇️  Downloading {out_path.name}")
    try:
        with urllib.request.urlopen(url) as r, open(out_path, "wb") as f:
            f.write(r.read())
        print(f"    ✅ {out_path.name} downloaded")
    except Exception as e:
        print(f"    ❌ Error downloading {out_path.name}: {e}")

# Download all required files
for fname, url in files_to_fetch.items():
    fetch_data(url, DATA_DIR / fname)

print("📊 All data files ready")



⬇️  Downloading integer_no_arrays.tsv
    ✅ integer_no_arrays.tsv downloaded
⬇️  Downloading integer_diet_quest_fields.tsv
    ✅ integer_diet_quest_fields.tsv downloaded
⬇️  Downloading integer_other_quest_fields.tsv
    ✅ integer_other_quest_fields.tsv downloaded
⬇️  Downloading real_fields1.tsv
    ✅ real_fields1.tsv downloaded
⬇️  Downloading real_fields2.tsv
    ✅ real_fields2.tsv downloaded
⬇️  Downloading string_fields1.tsv
    ✅ string_fields1.tsv downloaded
📊 All data files ready


In [22]:
FIELD_MAP = {
    "31": "sex",
    "21003": "age",
    "21001": "bmi",
    "48": "waist",
    "1558": "alcohol_freq",
    "20116": "smoking_status",
    "1160": "sleep_hrs",
    "874": "vigorous_mins",
    "864": "moderate_mins",
    "1309": "fruit_intake",
    "1319": "veg_intake",
    "1349": "red_meat_intake",
    "22009": "PC",  # Principal components for ancestry
}

In [17]:

def get_wanted_columns(header):
    """Extract relevant columns based on field IDs"""
    import re
    keep = set()
    for field_id in FIELD_MAP:
        pattern = re.compile(rf"^{field_id}-0\.\d+$")
        keep.update([c for c in header if pattern.match(c)])
    return keep | {"eid"}

def load_and_process_data():
    """Load and merge all TSV files"""
    print("📈 Loading and processing data...")

    frames = []
    for tfile in DATA_DIR.glob("*.tsv"):
        try:
            header = pd.read_csv(tfile, nrows=0, sep="\t").columns
            wanted_cols = get_wanted_columns(header)

            if len(wanted_cols) > 1:
                rename_dict = {}
                for col in wanted_cols:
                    if col == "eid":
                        continue
                    field_id = col.split('-')[0]
                    if field_id == "22009":
                        pc_num = int(col.split('.')[-1]) + 1
                        rename_dict[col] = f"PC{pc_num}"
                    else:
                        rename_dict[col] = FIELD_MAP.get(field_id, col)

                df = pd.read_csv(tfile, sep="\t", usecols=wanted_cols)
                df = df.set_index("eid").rename(columns=rename_dict)
                frames.append(df)
                print(f"    ✅ Processed {tfile.name}: {df.shape}")

        except Exception as e:
            print(f"    ⚠️  Error processing {tfile.name}: {e}")

    if frames:
        phenotype_data = pd.concat(frames, axis=1, join="inner").reset_index()
        print(f"📊 Combined dataset shape: {phenotype_data.shape}")
        return phenotype_data
    else:
        raise ValueError("No data frames were successfully loaded")



In [18]:
CATEGORY_MAPPINGS = {
    'sex': {0: 'Female', 1: 'Male'},
    'smoking_status': {0: 'Never', 1: 'Previous', 2: 'Current', 3: 'Unknown'},
    'alcohol_freq': {
        1: 'Daily', 2: 'Weekly_3_4', 3: 'Weekly_1_2',
        4: 'Monthly', 5: 'Occasional', 6: 'Never', 0: 'Unknown'
    }
}

In [ ]:
# Data Exploration and Visualization Functions

def explore_dataset(df):
    """Comprehensive data exploration"""
    print("\n=== Dataset Overview ===")
    print(f"Total samples: {len(df)}")
    print("\nMissing Values Summary:")
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Missing %': missing_pct
    }).query('`Missing Count` > 0')
    print(missing_df)
    
    print("\nNumerical Variables Summary:")
    print(df.describe().round(2))
    
    return df

def plot_distributions(df):
    """Plot distributions of key variables"""
    # Set up the plotting style
    plt.style.use('seaborn')
    
    # Create a figure with multiple subplots
    fig = plt.figure(figsize=(20, 15))
    
    # 1. Age Distribution
    plt.subplot(3, 3, 1)
    sns.histplot(data=df, x='age', bins=30)
    plt.title('Age Distribution')
    plt.axvline(df['age'].mean(), color='r', linestyle='--', label=f'Mean: {df["age"].mean():.1f}')
    plt.legend()
    
    # 2. BMI Distribution
    plt.subplot(3, 3, 2)
    sns.histplot(data=df, x='bmi', bins=30)
    plt.title('BMI Distribution')
    plt.axvline(25, color='g', linestyle='--', label='Overweight threshold')
    plt.axvline(30, color='r', linestyle='--', label='Obese threshold')
    plt.legend()
    
    # 3. Sex Distribution
    plt.subplot(3, 3, 3)
    sex_counts = df['sex'].map({0: 'Female', 1: 'Male'}).value_counts()
    plt.pie(sex_counts.values, labels=sex_counts.index, autopct='%1.1f%%')
    plt.title('Sex Distribution')
    
    # 4. Physical Activity
    plt.subplot(3, 3, 4)
    total_activity = df['moderate_mins'] + df['vigorous_mins'] * 2
    sns.histplot(data=total_activity[total_activity < 1000], bins=30)
    plt.title('Total Physical Activity (MET-minutes)')
    plt.axvline(150, color='g', linestyle='--', label='WHO Recommendation')
    plt.legend()
    
    # 5. Sleep Hours
    plt.subplot(3, 3, 5)
    sns.histplot(data=df, x='sleep_hrs', bins=30)
    plt.title('Sleep Hours Distribution')
    plt.axvline(7, color='g', linestyle='--', label='Recommended min')
    plt.axvline(9, color='r', linestyle='--', label='Recommended max')
    plt.legend()
    
    # 6. Smoking Status
    plt.subplot(3, 3, 6)
    smoking_counts = df['smoking_status'].map({
        0: 'Never', 1: 'Previous', 2: 'Current', 3: 'Unknown'
    }).value_counts()
    plt.bar(smoking_counts.index, smoking_counts.values)
    plt.title('Smoking Status Distribution')
    plt.xticks(rotation=45)
    
    # 7. Alcohol Frequency
    plt.subplot(3, 3, 7)
    alcohol_counts = df['alcohol_freq'].map({
        1: 'Daily', 2: 'Weekly_3_4', 3: 'Weekly_1_2',
        4: 'Monthly', 5: 'Occasional', 6: 'Never', 0: 'Unknown'
    }).value_counts()
    plt.bar(alcohol_counts.index, alcohol_counts.values)
    plt.title('Alcohol Consumption Frequency')
    plt.xticks(rotation=45)
    
    # 8. Diet (Fruit & Veg)
    plt.subplot(3, 3, 8)
    total_fruit_veg = df['fruit_intake'] + df['veg_intake']
    sns.histplot(data=total_fruit_veg[total_fruit_veg < 50], bins=30)
    plt.title('Total Fruit & Vegetable Intake')
    plt.axvline(5, color='g', linestyle='--', label='5-a-day target')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

def plot_correlations(df):
    """Plot correlation heatmap of numerical variables"""
    # Select numerical columns
    num_cols = ['age', 'bmi', 'waist', 'sleep_hrs', 'vigorous_mins', 
                'moderate_mins', 'fruit_intake', 'veg_intake']
    
    # Calculate correlations
    corr = df[num_cols].corr()
    
    # Create heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr, annot=True, cmap='coolwarm', center=0,
                fmt='.2f', square=True)
    plt.title('Correlation Matrix of Key Variables')
    plt.tight_layout()
    plt.show()

def analyze_risk_factors(df):
    """Analyze T2DM risk factors"""
    plt.figure(figsize=(15, 10))
    
    # 1. BMI vs T2DM
    plt.subplot(2, 2, 1)
    sns.boxplot(x='t2dm_event', y='bmi', data=df)
    plt.title('BMI Distribution by T2DM Status')
    plt.xlabel('T2DM Status (0=No, 1=Yes)')
    
    # 2. Age vs T2DM
    plt.subplot(2, 2, 2)
    sns.boxplot(x='t2dm_event', y='age', data=df)
    plt.title('Age Distribution by T2DM Status')
    plt.xlabel('T2DM Status (0=No, 1=Yes)')
    
    # 3. Physical Activity vs T2DM
    plt.subplot(2, 2, 3)
    total_activity = df['moderate_mins'] + df['vigorous_mins'] * 2
    activity_t2dm = pd.DataFrame({
        'T2DM': df['t2dm_event'],
        'Total Activity': total_activity
    })
    sns.boxplot(x='T2DM', y='Total Activity', data=activity_t2dm)
    plt.title('Physical Activity by T2DM Status')
    plt.xlabel('T2DM Status (0=No, 1=Yes)')
    
    # 4. Lifestyle Score vs T2DM
    plt.subplot(2, 2, 4)
    sns.boxplot(x='t2dm_event', y='lifestyle_score', data=df)
    plt.title('Lifestyle Score by T2DM Status')
    plt.xlabel('T2DM Status (0=No, 1=Yes)')
    
    plt.tight_layout()
    plt.show()

def plot_lifestyle_patterns(df):
    """Analyze and visualize lifestyle patterns"""
    fig = plt.figure(figsize=(15, 10))
    
    # 1. Physical Activity vs Sleep
    plt.subplot(2, 2, 1)
    total_activity = df['moderate_mins'] + df['vigorous_mins'] * 2
    plt.scatter(df['sleep_hrs'], total_activity, alpha=0.1)
    plt.title('Physical Activity vs Sleep Hours')
    plt.xlabel('Sleep Hours')
    plt.ylabel('Total Physical Activity (MET-minutes)')
    
    # 2. BMI vs Physical Activity
    plt.subplot(2, 2, 2)
    plt.scatter(total_activity, df['bmi'], alpha=0.1)
    plt.title('BMI vs Physical Activity')
    plt.xlabel('Total Physical Activity (MET-minutes)')
    plt.ylabel('BMI')
    
    # 3. Diet Quality
    plt.subplot(2, 2, 3)
    plt.scatter(df['fruit_intake'], df['veg_intake'], alpha=0.1)
    plt.title('Fruit vs Vegetable Intake')
    plt.xlabel('Fruit Intake (portions)')
    plt.ylabel('Vegetable Intake (portions)')
    
    # 4. Age vs Lifestyle Score
    plt.subplot(2, 2, 4)
    plt.scatter(df['age'], df['lifestyle_score'], alpha=0.1)
    plt.title('Age vs Lifestyle Score')
    plt.xlabel('Age')
    plt.ylabel('Lifestyle Score')
    
    plt.tight_layout()
    plt.show()

# Run the exploration and visualization
print("Starting comprehensive data exploration and visualization...")
df_explored = explore_dataset(df)
plot_distributions(df_explored)
plot_correlations(df_explored)
analyze_risk_factors(df_explored)
plot_lifestyle_patterns(df_explored)
print("Data exploration complete!")


In [ ]:
# Statistical Analysis and Risk Factor Assessment

def analyze_risk_groups(df):
    """Analyze T2DM risk across different demographic and lifestyle groups"""
    print("\n=== T2DM Risk Analysis ===")
    
    # 1. Age Groups
    df['age_group'] = pd.cut(df['age'], 
                            bins=[0, 45, 50, 55, 60, 65, 100],
                            labels=['<45', '45-50', '50-55', '55-60', '60-65', '65+'])
    
    age_risk = df.groupby('age_group')['t2dm_event'].agg(['mean', 'count'])
    age_risk['mean'] = (age_risk['mean'] * 100).round(2)
    print("\nT2DM Risk by Age Group:")
    print(age_risk)
    
    # 2. BMI Categories
    df['bmi_category'] = pd.cut(df['bmi'],
                               bins=[0, 18.5, 25, 30, 100],
                               labels=['Underweight', 'Normal', 'Overweight', 'Obese'])
    
    bmi_risk = df.groupby('bmi_category')['t2dm_event'].agg(['mean', 'count'])
    bmi_risk['mean'] = (bmi_risk['mean'] * 100).round(2)
    print("\nT2DM Risk by BMI Category:")
    print(bmi_risk)
    
    # 3. Lifestyle Score Groups
    df['lifestyle_group'] = pd.qcut(df['lifestyle_score'], q=4, 
                                  labels=['Poor', 'Fair', 'Good', 'Excellent'])
    
    lifestyle_risk = df.groupby('lifestyle_group')['t2dm_event'].agg(['mean', 'count'])
    lifestyle_risk['mean'] = (lifestyle_risk['mean'] * 100).round(2)
    print("\nT2DM Risk by Lifestyle Score:")
    print(lifestyle_risk)
    
    # Visualize the risk patterns
    plt.figure(figsize=(15, 5))
    
    # Age groups
    plt.subplot(1, 3, 1)
    plt.bar(age_risk.index, age_risk['mean'])
    plt.title('T2DM Risk by Age Group')
    plt.xlabel('Age Group')
    plt.ylabel('Risk (%)')
    plt.xticks(rotation=45)
    
    # BMI categories
    plt.subplot(1, 3, 2)
    plt.bar(bmi_risk.index, bmi_risk['mean'])
    plt.title('T2DM Risk by BMI Category')
    plt.xlabel('BMI Category')
    plt.ylabel('Risk (%)')
    plt.xticks(rotation=45)
    
    # Lifestyle score
    plt.subplot(1, 3, 3)
    plt.bar(lifestyle_risk.index, lifestyle_risk['mean'])
    plt.title('T2DM Risk by Lifestyle Score')
    plt.xlabel('Lifestyle Score Group')
    plt.ylabel('Risk (%)')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()
    
    return df

def analyze_combined_risk_factors(df):
    """Analyze interaction between multiple risk factors"""
    # Create a pivot table of BMI category and lifestyle score
    risk_matrix = pd.pivot_table(df, 
                                values='t2dm_event',
                                index='bmi_category',
                                columns='lifestyle_group',
                                aggfunc='mean')
    
    # Visualize the interaction
    plt.figure(figsize=(10, 8))
    sns.heatmap(risk_matrix * 100, annot=True, fmt='.1f', cmap='YlOrRd')
    plt.title('T2DM Risk (%) by BMI and Lifestyle Score')
    plt.tight_layout()
    plt.show()
    
    # Analyze physical activity patterns
    activity_groups = pd.qcut(df['moderate_mins'] + df['vigorous_mins'] * 2, 
                            q=4, labels=['Low', 'Moderate', 'High', 'Very High'])
    
    activity_bmi_risk = pd.pivot_table(df,
                                     values='t2dm_event',
                                     index='bmi_category',
                                     columns=activity_groups,
                                     aggfunc='mean')
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(activity_bmi_risk * 100, annot=True, fmt='.1f', cmap='YlOrRd')
    plt.title('T2DM Risk (%) by BMI and Physical Activity Level')
    plt.tight_layout()
    plt.show()

def calculate_risk_metrics(df):
    """Calculate various risk metrics and statistics"""
    print("\n=== Risk Factor Analysis ===")
    
    # Calculate odds ratios for key risk factors
    def calculate_odds_ratio(factor, threshold):
        high_risk = df[df[factor] >= threshold]['t2dm_event'].mean()
        low_risk = df[df[factor] < threshold]['t2dm_event'].mean()
        odds_ratio = (high_risk / (1 - high_risk)) / (low_risk / (1 - low_risk))
        return odds_ratio
    
    risk_factors = {
        'bmi': 30,  # Obesity threshold
        'age': 60,  # Age threshold
        'waist': 102,  # High waist circumference threshold
        'lifestyle_score': df['lifestyle_score'].median()  # Median lifestyle score
    }
    
    print("\nOdds Ratios for Risk Factors:")
    for factor, threshold in risk_factors.items():
        odds_ratio = calculate_odds_ratio(factor, threshold)
        print(f"{factor} (threshold: {threshold}): {odds_ratio:.2f}")
    
    # Calculate population attributable risk for obesity
    total_risk = df['t2dm_event'].mean()
    normal_weight_risk = df[df['bmi'] < 25]['t2dm_event'].mean()
    par = (total_risk - normal_weight_risk) / total_risk * 100
    
    print(f"\nPopulation Attributable Risk for Obesity: {par:.1f}%")

# Run the additional analyses
print("Performing detailed risk analysis...")
df_with_groups = analyze_risk_groups(df_explored)
analyze_combined_risk_factors(df_with_groups)
calculate_risk_metrics(df_with_groups)
print("Risk analysis complete!")


In [ ]:
# Model Performance Visualization and Interpretation

def plot_model_performance_comparison(results):
    """Visualize and compare model performances"""
    metrics = ['auc', 'f1']
    model_names = list(results.keys())
    
    plt.figure(figsize=(12, 5))
    
    # Plot AUC-ROC comparison
    plt.subplot(1, 2, 1)
    auc_scores = [results[model]['auc'] for model in model_names]
    plt.bar(model_names, auc_scores)
    plt.title('AUC-ROC Scores by Model')
    plt.xlabel('Model')
    plt.ylabel('AUC-ROC')
    plt.xticks(rotation=45)
    
    # Plot F1 scores comparison
    plt.subplot(1, 2, 2)
    f1_scores = [results[model]['f1'] for model in model_names]
    plt.bar(model_names, f1_scores)
    plt.title('F1 Scores by Model')
    plt.xlabel('Model')
    plt.ylabel('F1 Score')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()

def plot_roc_curves(results, X_test, y_test):
    """Plot ROC curves for all models"""
    plt.figure(figsize=(10, 8))
    
    for name, result in results.items():
        fpr, tpr, _ = roc_curve(y_test, result['prob'])
        plt.plot(fpr, tpr, label=f'{name} (AUC = {result["auc"]:.3f})')
    
    plt.plot([0, 1], [0, 1], 'k--', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves for All Models')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

def analyze_feature_importance(results, feature_names):
    """Analyze and visualize feature importance"""
    # Get the best model
    best_model_name = max(results.keys(), key=lambda k: results[k]['auc'])
    best_model = results[best_model_name]['pipeline'].named_steps['clf']
    
    # Get feature importance (if available)
    if hasattr(best_model, 'feature_importances_'):
        importance = best_model.feature_importances_
        
        # Sort features by importance
        feature_imp = pd.DataFrame({
            'feature': feature_names,
            'importance': importance
        }).sort_values('importance', ascending=False)
        
        # Plot feature importance
        plt.figure(figsize=(12, 6))
        sns.barplot(data=feature_imp.head(15), x='importance', y='feature')
        plt.title(f'Top 15 Most Important Features ({best_model_name})')
        plt.xlabel('Feature Importance')
        plt.tight_layout()
        plt.show()
        
        print("\nTop 10 Most Important Features:")
        print(feature_imp.head(10))

def analyze_prediction_thresholds(results, y_test):
    """Analyze the effect of different prediction thresholds"""
    # Get the best model
    best_model_name = max(results.keys(), key=lambda k: results[k]['auc'])
    best_probs = results[best_model_name]['prob']
    
    thresholds = np.arange(0.1, 0.9, 0.1)
    metrics = []
    
    for threshold in thresholds:
        preds = (best_probs >= threshold).astype(int)
        precision = precision_score(y_test, preds)
        recall = recall_score(y_test, preds)
        f1 = f1_score(y_test, preds)
        
        metrics.append({
            'threshold': threshold,
            'precision': precision,
            'recall': recall,
            'f1': f1
        })
    
    metrics_df = pd.DataFrame(metrics)
    
    # Plot metrics vs threshold
    plt.figure(figsize=(10, 6))
    plt.plot(metrics_df['threshold'], metrics_df['precision'], label='Precision')
    plt.plot(metrics_df['threshold'], metrics_df['recall'], label='Recall')
    plt.plot(metrics_df['threshold'], metrics_df['f1'], label='F1')
    plt.xlabel('Classification Threshold')
    plt.ylabel('Score')
    plt.title('Performance Metrics vs Classification Threshold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

# Run model performance analysis
print("Analyzing model performance...")
plot_model_performance_comparison(results)
plot_roc_curves(results, X_test, y_test)
analyze_feature_importance(results, X.columns)
analyze_prediction_thresholds(results, y_test)
print("Model analysis complete!")


In [19]:
def clean_and_preprocess(df):
    """Clean sentinel values, preprocess numerics, encode categoricals as int codes."""
    print("🧹 Cleaning and preprocessing data...")

    SENTINELS = {-1, -3, -7, -8, -9, -999, -9999, -99999,
                 -999999, -9999999, -99999999, -999999999, -9999999999}

    numeric_cols = df.select_dtypes(include=[np.number]).columns.difference(['eid'])
    df[numeric_cols] = df[numeric_cols].replace(list(SENTINELS), np.nan)
    df[numeric_cols] = df[numeric_cols].mask(df[numeric_cols] < -1e8, np.nan)

    if 'age' in df.columns:
        df = df.query("40 <= age <= 69").reset_index(drop=True)
        print(f"    After age filtering: {df.shape[0]} participants")

    # ------- NEW: keep categoricals as integer codes ----------
    if 'sex' in df.columns:
        df['sex'] = df['sex'].fillna(0).astype(int)                 # 0=Female, 1=Male

    if 'smoking_status' in df.columns:
        df['smoking_status'] = df['smoking_status'].fillna(3).astype(int)  # 3=Unknown

    if 'alcohol_freq' in df.columns:
        df['alcohol_freq'] = df['alcohol_freq'].fillna(0).astype(int)      # 0=Unknown
    # ----------------------------------------------------------

    if 'red_meat_intake' in df.columns:
        redmeat_map = {0:0.0, 1:0.5, 2:1.0, 3:3.0, 4:5.5, 5:7.0}
        df['red_meat_servings'] = df['red_meat_intake'].map(redmeat_map)
        df['red_meat_missing']  = df['red_meat_intake'].isna().astype(int)

    for col in ['moderate_mins', 'vigorous_mins']:
        if col in df.columns:
            df[f'{col}_missing'] = df[col].isna().astype(int)
            df[col] = df[col].fillna(0)

    print(f"    ✅ Data cleaned. Final shape: {df.shape}")
    return df





def create_enhanced_features(df):
    """Enhanced feature engineering with lifestyle scores and PGS proxy"""
    print("🔧 Creating enhanced features...")

    # Lifestyle Score Components
    lifestyle_score = 0

    # BMI score (0-2 points)
    if 'bmi' in df.columns:
        bmi_filled = df['bmi'].fillna(df['bmi'].median())
        df['bmi_score'] = np.where(bmi_filled < 25, 2,
                         np.where(bmi_filled < 30, 1, 0))
        lifestyle_score += df['bmi_score']

    # Physical activity score (0-2 points)
    if 'moderate_mins' in df.columns and 'vigorous_mins' in df.columns:
        total_activity = df['moderate_mins'] + df['vigorous_mins'] * 2
        df['activity_score'] = np.where(total_activity >= 150, 2,
                              np.where(total_activity >= 75, 1, 0))
        lifestyle_score += df['activity_score']

    # Smoking score (0-2 points)
    if 'smoking_status' in df.columns:
        df['smoking_score'] = np.where(df['smoking_status'] == 0, 2,
                             np.where(df['smoking_status'] == 1, 1, 0))
        lifestyle_score += df['smoking_score']

    # Diet score (0-2 points) - simplified
    if 'fruit_intake' in df.columns and 'veg_intake' in df.columns:
        fruit_filled = df['fruit_intake'].fillna(df['fruit_intake'].median())
        veg_filled = df['veg_intake'].fillna(df['veg_intake'].median())
        diet_score = (fruit_filled + veg_filled) / 2
        df['diet_score'] = np.where(diet_score > df['fruit_intake'].median(), 2,
                          np.where(diet_score > df['fruit_intake'].median() * 0.5, 1, 0))
        lifestyle_score += df['diet_score']

    # Sleep score (0-2 points)
    if 'sleep_hrs' in df.columns:
        sleep_filled = df['sleep_hrs'].fillna(df['sleep_hrs'].median())
        df['sleep_score'] = np.where((sleep_filled >= 7) & (sleep_filled <= 9), 2,
                           np.where((sleep_filled >= 6) & (sleep_filled <= 10), 1, 0))
        lifestyle_score += df['sleep_score']

    df['lifestyle_score'] = lifestyle_score

    # Polygenic Risk Score proxy using PCs
    pc_cols = [col for col in df.columns if col.startswith('PC')]
    if len(pc_cols) >= 5:
        # Create PGS proxy from first 5 PCs with random weights
        np.random.seed(RANDOM_STATE)
        weights = np.random.normal(0, 0.1, 5)
        pgs_proxy = 0
        for i, pc_col in enumerate(pc_cols[:5]):
            if pc_col in df.columns:
                pc_norm = df[pc_col].fillna(0) / (df[pc_col].std() + 1e-8)
                pgs_proxy += pc_norm * weights[i]
        df['pgs_proxy'] = pgs_proxy

    # Age groups for fairness analysis
    if 'age' in df.columns:
        df['age_group'] = pd.cut(df['age'], bins=[40, 50, 60, 70],
                                labels=['40-50', '50-60', '60-70'])

    print(f"    ✅ Enhanced features created. Shape: {df.shape}")
    return df





def create_target_variable(df, prevalence=0.07):
    """Create enhanced synthetic T2DM outcome"""
    print(f"🎯 Creating enhanced T2DM target variable (prevalence: {prevalence:.1%})")

    risk_score = np.zeros(len(df))

    # Enhanced risk modeling
    if 'age' in df.columns:
        age_norm = (df['age'] - df['age'].mean()) / df['age'].std()
        risk_score += age_norm * 0.8

    if 'bmi' in df.columns:
        bmi_filled = df['bmi'].fillna(df['bmi'].median())
        risk_score += np.where(bmi_filled >= 30, 1.5, 0)
        risk_score += np.where(bmi_filled >= 25, 0.8, 0)

    if 'waist' in df.columns:
        waist_filled = df['waist'].fillna(df['waist'].median())
        risk_score += np.where(waist_filled > 102, 0.7, 0)
        risk_score += np.where(waist_filled > 88, 0.5, 0)

    # Use lifestyle score (protective)
    if 'lifestyle_score' in df.columns:
        lifestyle_norm = (df['lifestyle_score'] - df['lifestyle_score'].mean()) / df['lifestyle_score'].std()
        risk_score -= lifestyle_norm * 0.6

    # Use PGS proxy
    if 'pgs_proxy' in df.columns:
        risk_score += df['pgs_proxy'] * 0.4

    if 'sex' in df.columns:
        risk_score += np.where(df['sex'] == 'Male', 0.3, 0)

    # Convert to probabilities
    logits = risk_score * 1.5  # sharper separation for better distinction
    probabilities = 1 / (1 + np.exp(-logits))

    current_mean = probabilities.mean()
    probabilities = probabilities * (prevalence / current_mean)
    probabilities = np.clip(probabilities, 0.001, 0.999)

    np.random.seed(RANDOM_STATE)
    df['t2dm_event'] = np.random.binomial(1, probabilities)

    actual_prevalence = df['t2dm_event'].mean()
    print(f"    ✅ T2DM cases: {df['t2dm_event'].sum()}/{len(df)} ({actual_prevalence:.1%})")

    return df

    def prepare_features(df):
    # No age filtering here
      df = df.copy()

      # Drop rows with missing target or key features
      df = df.dropna(subset=['age'])

      # Age binning (optional)
      df['age_bin'] = pd.cut(df['age'], bins=[0, 30, 40, 50, 60, 70, 80, 100], labels=False)

      # Identify feature types
      numeric_features = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
      numeric_features = [col for col in numeric_features if col not in ['T2DM', 'age']]
      categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist() + ['age_bin']

      # Pipelines
      numeric_transformer = Pipeline(steps=[
          ('imputer', SimpleImputer(strategy='median')),
          ('scaler', StandardScaler())
      ])
      categorical_transformer = Pipeline(steps=[
          ('imputer', SimpleImputer(strategy='most_frequent')),
          ('onehot', OneHotEncoder(handle_unknown='ignore'))
      ])

      preprocessor = ColumnTransformer(
          transformers=[
              ('num', numeric_transformer, numeric_features),
              ('cat', categorical_transformer, categorical_features)
          ])

      X = df.drop(columns=['T2DM'])
      y = df['T2DM']

      return X, y, preprocessor



def prepare_features_enhanced(df):
    """Enhanced feature preparation with PCA"""
    print("🔧 Preparing enhanced features...")

    if 't2dm_event' not in df.columns:
        df = create_target_variable(df)

    y = df['t2dm_event']
    exclude_cols = ['eid', 't2dm_event', 'age_group']
    X = df.drop(columns=exclude_cols)

    # mark categorical int columns
    cat_int_cols = ['sex', 'smoking_status', 'alcohol_freq']

    # treat everything (including int-coded cats) as numeric for modelling
    numeric_features = list(X.columns)
    categorical_features = cat_int_cols             # for SMOTENC only

    print(f"    📊 Numeric features ({len(numeric_features)}) "
          f"| SMOTENC categorical ints ({len(categorical_features)})")
    return X, y, numeric_features, categorical_features, df.get('age_group')

def create_preprocessing_pipeline_enhanced(numeric_features, _categorical_unused):
    """Simple numeric pipeline (+ optional PCA for PCs)."""
    print("⚙️ Creating enhanced preprocessing pipeline...")

    pc_features    = [c for c in numeric_features if c.startswith('PC')]
    other_numeric  = [c for c in numeric_features if c not in pc_features]

    pc_pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler',  StandardScaler()),
        ('pca',     PCA(n_components=min(10, len(pc_features)), random_state=RANDOM_STATE))
    ]) if pc_features else None

    num_pipe = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler',  StandardScaler())
    ])

    transformers = [('num', num_pipe, other_numeric)]
    if pc_pipe:
        transformers.append(('pc', pc_pipe, pc_features))

    return ColumnTransformer(transformers)

from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
from collections import Counter

from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTENC
from collections import Counter

def train_enhanced_models(X, y, preprocessor, age_groups=None):
    print("🚀 Training enhanced ensemble models...")

    # split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=RANDOM_STATE)

    # int-coded categorical indices for SMOTENC
    cat_cols = ['sex', 'smoking_status', 'alcohol_freq']
    cat_idx  = [i for i, c in enumerate(X.columns) if c in cat_cols]

    # encode NaNs to sentinel for SMOTENC
    X_train_sm = X_train.fillna(-999)
    X_test_sm  = X_test.fillna(-999)

    smote = SMOTENC(categorical_features=cat_idx, random_state=RANDOM_STATE)

    models = {
    # ▸ Linear baseline
    'LogReg': LogisticRegression(
        max_iter=1000,
        class_weight='balanced',
        random_state=RANDOM_STATE
    ),

    # ▸ Classical tree
    'RandomForest': RandomForestClassifier(
        n_estimators=300,
        max_depth=12,
        min_samples_split=10,
        class_weight='balanced',
        random_state=RANDOM_STATE,
        n_jobs=-1
    ),

    # ▸ Gradient-boosted trees (LightGBM)
    'LightGBM': lgb.LGBMClassifier(
        # -- GPU acceleration if your Colab has it and LightGBM-GPU is installed
        #    comment out device='gpu' if your build is CPU-only
        device='gpu',
        boosting_type='gbdt',
        n_estimators=300,
        learning_rate=0.05,
        max_depth=-1,
        num_leaves=64,
        class_weight='balanced',
        scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),
        random_state=RANDOM_STATE
    ),

    # ▸ GPU-accelerated XGBoost
    'XGB': xgb.XGBClassifier(
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        gpu_id=0,
        n_estimators=300,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.8,
        scale_pos_weight=(y_train == 0).sum() / (y_train == 1).sum(),
        eval_metric='logloss',
        random_state=RANDOM_STATE
    )
    }
    results = {}
    for name, clf in models.items():
        print(f"\n📈 {name}")
        pipe = ImbPipeline([
            ('smote', smote),
            ('prep',  preprocessor),
            ('clf',   clf)
        ])
        pipe.fit(X_train_sm, y_train)

        prob = pipe.predict_proba(X_test_sm)[:,1]
        pred = (prob >= 0.15).astype(int)
        results[name] = {
              'pipeline': pipe,
              'auc': roc_auc_score(y_test, prob),
              'f1' : f1_score(y_test, pred),
              'prob': prob,
              'pred': pred
          }
        print(f"    AUC={results[name]['auc']:.3f} | F1={results[name]['f1']:.3f}")

    return results, X_test_sm, y_test

def decode_column(series, col_name):
    """Convert int codes back to readable labels for plots."""
    return series.map(CATEGORY_MAPPINGS[col_name]).astype('category')

def evaluate_fairness(results, X_test, y_test, age_groups=None):
    """Basic fairness evaluation across demographics"""
    print("\n⚖️ Evaluating model fairness...")

    if age_groups is None:
        print("    No demographic data available for fairness analysis")
        return

    best_model_name = max(results.keys(), key=lambda k: results[k]['auc_roc'])
    best_model_results = results[best_model_name]
    y_pred_proba = best_model_results['y_pred_proba']

    print(f"\n    Fairness analysis for best model: {best_model_name}")

    # Group-wise performance
    for group in age_groups.unique():
        if pd.isna(group):
            continue
        mask = age_groups == group
        if mask.sum() < 10:  # Skip small groups
            continue

        group_auc = roc_auc_score(y_test[mask], y_pred_proba[mask])
        group_prev = y_test[mask].mean()

        print(f"    {group}: AUC-ROC = {group_auc:.3f}, Prevalence = {group_prev:.1%}, N = {mask.sum()}")

def basic_shap_analysis(results, X_train, X_test, feature_names=None):
    """Basic SHAP analysis for interpretability"""
    print("\n🔍 Performing basic SHAP analysis...")

    try:
        # Get best model
        best_model_name = max(results.keys(), key=lambda k: results[k]['auc_roc'])
        best_pipeline = results[best_model_name]['pipeline']

        print(f"    Analyzing {best_model_name}...")

        # Transform data for SHAP
        X_train_transformed = best_pipeline.named_steps['base_estimator'].named_steps['preprocessor'].transform(X_train[:1000])  # Sample for speed
        X_test_transformed = best_pipeline.named_steps['base_estimator'].named_steps['preprocessor'].transform(X_test[:100])

        # Get the actual classifier
        classifier = best_pipeline.named_steps['base_estimator'].named_steps['classifier']

        # Create SHAP explainer (basic)
        if hasattr(classifier, 'predict_proba'):
            explainer = shap.Explainer(classifier.predict_proba, X_train_transformed)
            shap_values = explainer(X_test_transformed)

            # Basic summary
            print(f"    ✅ SHAP analysis completed for {X_test_transformed.shape[0]} samples")
            print(f"    Feature importance calculated with {X_train_transformed.shape[1]} features")

        else:
            print("    ⚠️  SHAP analysis not supported for this model type")

    except Exception as e:
        print(f"    ⚠️  SHAP analysis failed: {str(e)[:100]}...")

def plot_calibration_curve(results, y_test):
    """Plot calibration curves for model comparison"""
    print("\n📊 Plotting calibration curves...")

    plt.figure(figsize=(10, 6))

    for name, result in results.items():
        y_pred_proba = result['y_pred_proba']
        fraction_of_positives, mean_predicted_value = calibration_curve(
            y_test, y_pred_proba, n_bins=10
        )

        plt.plot(mean_predicted_value, fraction_of_positives,
                marker='o', label=f"{name} (Brier: {result['brier_score']:.3f})")

    plt.plot([0, 1], [0, 1], 'k--', label='Perfect Calibration')
    plt.xlabel('Mean Predicted Probability')
    plt.ylabel('Fraction of Positives')
    plt.title('Calibration Curves - T2DM Prediction Models')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()



In [20]:
def main():
    """Enhanced main execution function"""
    print("🏥 Enhanced Type 2 Diabetes Prediction Pipeline")
    print("=" * 60)

    try:
        # Load and process data
        df = load_and_process_data()
        df = clean_and_preprocess(df)

        # Enhanced feature engineering
        df = create_enhanced_features(df)
        df = create_target_variable(df)
        print("\n🔍 Sample of cleaned data:")
        display(df.head())

        print("\n📊 Class distribution preview:")
        print(df['t2dm_event'].value_counts(normalize=True).rename({0: 'Non-T2DM', 1: 'T2DM'}).round(3))


        # Enhanced feature preparation
        X, y, numeric_features, categorical_features, age_groups = prepare_features_enhanced(df)

        # Enhanced preprocessing
        preprocessor = create_preprocessing_pipeline_enhanced(numeric_features, categorical_features)

        # Enhanced model training
        results, X_train, X_test, y_train, y_test, age_test = train_enhanced_models(
            X, y, preprocessor, age_groups
        )

        # Enhanced evaluation
        print("\n🎉 Enhanced pipeline completed successfully!")
        print("\n📊 Enhanced Model Performance Summary:")
        print("-" * 50)

        for name, result in results.items():
            print(f"{name}:")
            print(f"  AUC-ROC: {result['auc_roc']:.3f}")
            print(f"  AUC-PR:  {result['auc_pr']:.3f}")
            print(f"  F1-Score: {result['f1_score']:.3f}")
            print(f"  Brier Score: {result['brier_score']:.3f}")
            print()

        # Fairness evaluation
        evaluate_fairness(results, X_test, y_test, age_test)

        # Basic interpretability
        basic_shap_analysis(results, X_train, X_test)

        # Calibration analysis
        plot_calibration_curve(results, y_test)

        return df, results

    except Exception as e:
        print(f"❌ Error in enhanced pipeline: {e}")
        import traceback
        traceback.print_exc()
        raise


In [ ]:
from imblearn.over_sampling import SMOTENC

if __name__ == "__main__":
    # Run the enhanced pipeline
    data, model_results = main()

🏥 Enhanced Type 2 Diabetes Prediction Pipeline
📈 Loading and processing data...
    ✅ Processed integer_no_arrays.tsv: (600000, 10)
    ✅ Processed real_fields1.tsv: (600000, 42)
📊 Combined dataset shape: (600000, 53)
🧹 Cleaning and preprocessing data...
    After age filtering: 360244 participants
    ✅ Data cleaned. Final shape: (360244, 57)
🔧 Creating enhanced features...
    ✅ Enhanced features created. Shape: (360244, 65)
🎯 Creating enhanced T2DM target variable (prevalence: 7.0%)
    ✅ T2DM cases: 25084/360244 (7.0%)

🔍 Sample of cleaned data:


,eid,sex,moderate_mins,vigorous_mins,sleep_hrs,fruit_intake,veg_intake,red_meat_intake,alcohol_freq,smoking_status,...,vigorous_mins_missing,bmi_score,activity_score,smoking_score,diet_score,sleep_score,lifestyle_score,pgs_proxy,age_group,t2dm_event
0,1000016,1,0.0,83.0,20.0,44.0,40.0,NaN,6,3,...,0,2,2,0,2,0,6,0.374418,50-60,0
1,1000048,0,4.0,480.0,8.0,6.0,24.0,NaN,1,2,...,0,1,2,0,1,2,6,-0.055388,40-50,1
2,1000059,0,0.0,35.0,9.0,22.0,1.0,NaN,1,3,...,0,1,0,0,1,2,4,-0.277568,NaN,0
3,1000063,0,-2.0,269.0,6.0,44.0,7.0,NaN,3,2,...,0,1,2,0,2,1,6,-0.073158,50-60,0
4,1000080,0,1.0,610.0,17.0,15.0,100.0,NaN,5,3,...,0,1,2,0,2,0,5,0.166802,60-70,0



📊 Class distribution preview:
t2dm_event
Non-T2DM    0.93
T2DM        0.07
Name: proportion, dtype: float64
🔧 Preparing enhanced features...
    📊 Numeric features (63) | SMOTENC categorical ints (3)
⚙️ Creating enhanced preprocessing pipeline...
🚀 Training enhanced ensemble models...

📈 LogReg
    AUC=0.580 | F1=0.144

📈 RandomForest
    AUC=0.593 | F1=0.150

📈 LightGBM
[LightGBM] [Info] Number of positive: 268128, number of negative: 268128
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 5185
[LightGBM] [Info] Number of data points in the train set: 536256, number of used features: 33
[LightGBM] [Info] Using GPU Device: NVIDIA A100-SXM4-40GB, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 27 dense feature groups (14.32 MB) transferred to GPU in 0.014335 secs. 1 sparse feature groups
[LightGBM] [Info] [binary